In [2]:
import re
import PyPDF2

from IPython.display import HTML
from textwrap import fill
from bs4 import BeautifulSoup

## Fixed-size Chunking

- Fixed-size chunking divides text into segments of a set size, no matter the structure of the content. 
- This can be enhanced by employing overlapping chunks, which enable neighboring chunks to have some content in common.
- This approach is the cheapest among other approaches.

In [ ]:
def read_pdf_in_chunks(file_path, chunk_size=1024):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        num_pages = len(reader.pages)
        for page_num in range(num_pages):
            page = reader.pages[page_num]
            text = page.extract_text()
            for i in range(0, len(text), chunk_size):
                yield text[i:i + chunk_size]

# Example usage:
file_path = './sample_k_means.pdf'
for chunk in read_pdf_in_chunks(file_path, chunk_size=1024):
    # Format and print chunk with word wrapping
    
    print(fill(chunk, width=100))
    def highlight_chunks(text, chunk_size=100):
        # Define colors for alternating highlights
        colors = ['#FFB6C1', '#90EE90', '#87CEEB', '#FFFF00']  # pink, green, blue, yellow
        
        soup = BeautifulSoup('<div></div>', 'html.parser')
        div = soup.div
        
        # Split text into chunks and highlight each with different colors
        for i in range(0, len(text), chunk_size):
            chunk = text[i:i + chunk_size]
            color = colors[(i//chunk_size) % len(colors)]
            span = soup.new_tag('span', style=f'background-color: {color}')
            span.string = chunk
            div.append(span)
        
        return str(soup)

    # Apply highlighting to chunks
    highlighted_text = highlight_chunks(chunk)
    display(HTML(f'<div style="width: {800}px">{highlighted_text}</div>'))

K-Means Clustering K-Means Clustering is an unsupervised learning method. If data set samples have
labels, we prefer to use supervised method, but in real world mostly we don’t have labels and that’s
why we prefer clustering methods which are known as unsupervised methods. The goal of this algorithm
is to find groups in the data, with the number of groups represented by the variable K. The
algorithm works iteratively to assign each data point to one of K groups based on the features that
are provided. Data points are clustered based on feature similarity. The results of the K-means
clustering algorithm are: The centroids of the K clusters, which can be used to label new data
Labels for the training data (each data point is assigned to a single cluster) Rather than defining
groups before looking at the data, clustering allows you to find and analyze the groups that have
been formed organically. The “Choosing K” section below describes how the number of groups can be
determined. Each cen

collection of feature values which define the resulting groups. Examining the centroid feature
weights can be used to qualitatively interpret what kind of group each cluster represents[1].
Project Details As, I have mentioned previously I used OpenCV library for image processing with
Python3 for this project. The other libraries that I have used NumPy for numerical arrays,
Matplotlib for visualizing my result, Sklearn for machine learning. First we will read image data
using cv2.imread() function from OpenCV as cv2. Once the image is read using cv2, our image color
channel comes to us as Blue-Green-Red. But we want Red-Green-Blue as our image color channel, so we
convert it to the required channel using cv2.cvtcolor() function. Now we have 3-D parameters in our
image data: row number X column number X colour channel number. But, we won’t need row and column
information separately. Besides, it is hard to deal with 3-D Matrix that’s why we reshape() image
and make it 2-D Matrix data. Thi

rt for images and now we are ready to go on with clustering. Since we will import K-Means from top
of our code, we can easily use it by only giving n_clusters, which represents cluster number
originally. After that we will use fit() function to apply K-Means Clustering algorithm on our pre-
processed image data and result will come back to clt objects. We use find_histogram() function to
limit the number of histograms to the desired number of clusters. You can find the full details
about find_histogram() and plot_colors2() function in this article below. As we don’t want to find
histogram for all pixel, and all the color palette, so we would like to limit it to the desired
number of clusters. The plot_colors2() function will prepare a bar, and put colors on this bar as
shown in the image below: Figure 1. dominant colour bar A histogram is an accurate graphical
representation of the distribution of numerical data. It is an estimate of the probability
distribution of a continuous variabl

istogram() function it takes one parameter which is trained clt object and returns histogram
information. np. arange(0, len(np.unique(clt.labels_))+1) function will return evenly spaced values
within a given interval. For our example, because we have 3 clusters numLabels’ value will be [0 1 2
3]. After that, we will calculate the histogram according to numberLabels and cluster’s label
values. clt.labels_ is an array which includes cluster labels for each pixel. After calculating the
histogram value, we will calculate the ratio of the histogram values we obtained because it is more
difficult to work with numbers than to work with ratios. After calculating the histogram ratios, we
will jump in to plot_colors2() function. This function will plot a bar which shows colours according
to clusters and histogram. First we prepare a 3-D matrix, all values are zero. Then we plot the
relative percentage of each cluster. cv2.rectangle() function draws a simple, thick, or filled up-
right rectangle.

le with colors in terms of histogram ratios. Below you can find all the code, I have mentioned above
or Github version can be found here.


In the end, we prepare an image processing application which aims to find the dominant color for an
image. We have used K-Means Clustering and calculate histogram to find the following result:


## Recursive Chunking
- More advanced approach than the fixed-size chunking. 
- Recursive chunking first divides text with a main separator (such as paragraphs) and subsequently utilizing secondary separators (like sentences) if the resulting chunks remain too large. 

In [3]:
def recursive_highlight_chunks(text, min_chunk_size=50, max_depth=10):
    colors = ['#FFB6C1', '#90EE90', '#87CEEB', '#FFFF00']
    
    def split_at_sentence(text, start_idx, end_idx):
        search_text = text[start_idx:end_idx]
        sentences = re.split(r'([.!?]+\s)', search_text)
        
        if len(sentences) <= 1:
            return end_idx
        
        total_length = end_idx - start_idx
        target_length = total_length // 2
        
        current_length = 0
        for i in range(0, len(sentences)-1, 2):
            sentence = sentences[i] + (sentences[i+1] if i+1 < len(sentences) else '')
            if current_length + len(sentence) >= target_length:
                return start_idx + current_length
            current_length += len(sentence)
        
        return start_idx + current_length

    def chunk_recursively(text, start, end, depth=0):
        if depth >= max_depth or end - start <= min_chunk_size:
            return [(start, end)]
        
        mid = split_at_sentence(text, start, end)
        if mid == end or mid == start:
            return [(start, end)]
        
        return chunk_recursively(text, start, mid, depth + 1) + \
               chunk_recursively(text, mid, end, depth + 1)
    
    chunks = chunk_recursively(text, 0, len(text))
    
    soup = BeautifulSoup('<div></div>', 'html.parser')
    div = soup.div
    
    for i, (start, end) in enumerate(chunks):
        chunk = text[start:end]
        color = colors[i % len(colors)]
        span = soup.new_tag('span', style=f'background-color: {color}')
        span.string = chunk
        div.append(span)
    
    return str(soup)

# Display the recursively highlighted text for all chunks
for chunk in read_pdf_in_chunks(file_path, chunk_size=1024):
    recursive_highlighted = recursive_highlight_chunks(chunk)
    display(HTML(f'<div style="width: {800}px">{recursive_highlighted}</div>'))


## Document-based Chunking

- This approach creates segments according to the inherent divisions in a document, such as headings, tags or paragraphs. 
- It works especially well for structured data formats like HTML, Markdown, or code files but is not as effective when the data does not contain distinct structural components.
- Below example has been chunked by following elements from an HTML file: `'p', 'h3', 'meta', 'span'`

In [17]:
def document_based_chunking(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')
    
    chunks = []
    current_chunk = []
    
    for element in soup.body.descendants:
        if element.name in ['p', 'h3', 'meta', 'span']:
            if current_chunk:
                chunks.append(' '.join(current_chunk))
                current_chunk = []
            current_chunk.append(element.get_text())
    
    if current_chunk:
        chunks.append(' '.join(current_chunk))
    
    return chunks

def highlight_chunks(chunks):
    colors = ['#FFB6C1', '#90EE90', '#87CEEB', '#FFFF00']
    soup = BeautifulSoup('<div></div>', 'html.parser')
    div = soup.div
    
    for i, chunk in enumerate(chunks):
        color = colors[i % len(colors)]
        span = soup.new_tag('span', style=f'background-color: {color}')
        span.string = chunk
        div.append(span)
    
    return str(soup)

# Example usage:
file_path = 'Bilge Ince.html'
chunks = document_based_chunking(file_path)
highlighted_chunks = highlight_chunks(chunks)
display(HTML(f'<div style="width: {800}px">{highlighted_chunks}</div>'))

## Semantic Chunking

- Semantic chunking segments text into meaningful components that are subsequently transformed into vectors. 
- These components are then grouped into chunks depending on the cosine distance of their embeddings, with a new chunk created each time a notable shift in context is identified. 
- This approach leverages from semantic search to decide the size of the chunks. It's more intelligent approach from above approaches but less cost effective.

In [3]:
from sentence_transformers import SentenceTransformer, util

def semantic_chunking(sentences, threshold=0.7):
    # Load pre-trained model
    model = SentenceTransformer('all-MiniLM-L6-v2')
    
    # Generate embeddings for each sentence
    embeddings = model.encode(sentences)
    
    # Calculate cosine similarity matrix
    cosine_sim_matrix = util.pytorch_cos_sim(embeddings, embeddings)

    # Group tokens into chunks based on cosine similarity
    groups = []
    visited = set()
    for i in range(len(sentences)):
        if i not in visited:
            group = [sentences[i]]
            visited.add(i)
            for j in range(i + 1, len(sentences)):
                if cosine_sim_matrix[i][j] > threshold and j not in visited:
                    group.append(sentences[j])
                    visited.add(j)
            groups.append(group)
    
    return groups

def highlight_chunks(chunks):
    colors = ['#FFB6C1', '#90EE90', '#87CEEB', '#FFFF00']
    soup = BeautifulSoup('<div></div>', 'html.parser')
    div = soup.div
    
    for i, chunk in enumerate(chunks):
        color = colors[i % len(colors)]
        span = soup.new_tag('span', style=f'background-color: {color}')
        span.string = chunk[0]
        div.append(span)
    
    return str(soup)

def read_pdf_in_sentences(file_path):
    sentences = []
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        num_pages = len(reader.pages)
        for page_num in range(num_pages):
            page = reader.pages[page_num]  
            text = page.extract_text()
            sentences.extend(re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', text))
    return sentences

# Example usage:
file_path = './sample_k_means.pdf'
sentences = read_pdf_in_sentences(file_path)
semantic_chunks = semantic_chunking(sentences)
highlighted_semantic_chunks = highlight_chunks(semantic_chunks)
display(HTML(f'<div style="width: {800}px">{highlighted_semantic_chunks}</div>'))

## LLM based Chunking

- Using LLM to generate chunks by processing text and creating semantically isolated sentences or propositions.
- Please note that this will be the most expensive way of chunking.

In [20]:

import openai
import backoff
import os

@backoff.on_exception(backoff.expo, openai.RateLimitError)
def _api_call(messages, model, max_tokens):
    client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    
    return client.chat.completions.create(
        model=model, 
        messages=messages,
        max_tokens=max_tokens,
        n=1,
        stop=None,
        temperature=0.5,
    )

def llm_chunking(text, model="gpt-3.5-turbo", max_tokens=500):
     # Replace with your OpenAI API key
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": f"Chunk the following text into semantically isolated sentences or propositions in a list:\n\n{text}\n\nChunks:",
        }
    ]
    completion = _api_call(messages, model=model, max_tokens=max_tokens)
    chunks = completion.choices[0].message.content.strip().split('\n')
    return chunks

def highlight_chunks(chunks):
    colors = ['#FFB6C1', '#90EE90', '#87CEEB', '#FFFF00']
    soup = BeautifulSoup('<div></div>', 'html.parser')
    div = soup.div
    
    for i, chunk in enumerate(chunks):
        color = colors[i % len(colors)]
        span = soup.new_tag('span', style=f'background-color: {color}')
        span.string = chunk
        div.append(span)
    
    return str(soup)

# Example usage:
file_path = './sample_k_means.pdf'
text = read_pdf_in_sentences(file_path)
llm_chunks = llm_chunking(text)
highlighted_semantic_chunks = highlight_chunks(llm_chunks)
display(HTML(f'<div style="width: {800}px">{highlighted_semantic_chunks}</div>'))
